# Day 4 - Graphene!

Today we are going to apply what we have learned and explore the electronic structure of graphene.

In this morning session,
- write script to perform scf calculation for graphene
    - calculate the band structure
    - calculate the density of states
- post processing!
    - plot results

## DFTK.jl

**Goal**: Write a script that take as input:

   - lattice,
   - atomic positions,
   - input parameters (Ecut, kgrid, etc.)

In [1]:
# load dependencies
using Dates
using DFTK
using Plots

In [ ]:
function run(lattice, atoms; extras)
    # setup of the DFT calculation

    # call self_consistent_field
    
    return scfres
end

In [2]:
# a simple solution
function run(lattice, atoms; Ecut=20, kgrid=[20, 20, 1], tol=1e-10, temperature=0.01)
    @info "Building DFT model"
    model = model_LDA(lattice, atoms, temperature=temperature)
    @info "Constructing plane-wave basis set"
    basis = PlaneWaveBasis(model, Ecut; kgrid=kgrid)
    @info "Solving Kohn-Sham equations self-consistently"
    println()
    println("Starting calculation at $(Dates.now())")
    println()
    println("no. of electrons    = $(model.n_electrons)")
    println("k grid              = $(kgrid)")
    println("temperature         = $(temperature)")
    println("Ecut                = $(Ecut)")
    @time scfres = self_consistent_field(basis; tol=tol)
    println()
    println("Finished calculation at $(Dates.now())")
    println()
    return basis, scfres
end;

In [5]:
a = 4.6111
c = 18.870
lattice = [[a -0.5*a 0]; [0 0.5*sqrt(3)*a 0]; [0 0 c]];
positions = [[1/3., 2/3., 0], [2/3., 1/3., 0]];
C = ElementPsp(:C, psp=load_psp(:C, functional="lda"));
atoms = [C => [pos for pos in positions]];

In [6]:
basis, scfres = run(lattice, atoms);

┌ Info: Building DFT model
└ @ Main In[2]:3
┌ Info: Constructing plane-wave basis set
└ @ Main In[2]:5



Starting calculation at 2021-07-01T08:51:27.354

no. of electrons    = 8


┌ Info: Solving Kohn-Sham equations self-consistently
└ @ Main In[2]:7


k grid              = [20, 20, 1]
temperature         = 0.01
Ecut                = 20
n     Free energy       Eₙ-Eₙ₋₁     ρout-ρin   Diag
---   ---------------   ---------   --------   ----
  1   -11.28756389586         NaN   2.62e-01    3.3 
  2   -11.29647393224   -8.91e-03   3.78e-02    2.2 
  3   -11.29707828927   -6.04e-04   8.04e-03    2.6 
  4   -11.29711399926   -3.57e-05   7.75e-04    2.6 
  5   -11.29711479756   -7.98e-07   1.18e-04    3.9 
  6   -11.29711481336   -1.58e-08   1.27e-05    3.4 
  7   -11.29711481357   -2.07e-10   1.56e-06    3.8 
  8   -11.29711481357   -2.37e-12   1.62e-07    3.5 
162.287322 seconds (113.67 M allocations: 18.185 GiB, 3.48% gc time)

Finished calculation at 2021-07-01T08:54:10.116



In [7]:
ρ = heatmap(scfres.ρ.real[:, :, 1, 1], c=:sunset);

In [8]:
bands = plot_bandstructure(scfres, kline_density=10)
plot!(bands, xlabel = " ", ylabel="ε - εF", ylims=(-10, 10));

Computing bands along kpath:
       Γ -> M -> K -> Γ -> A -> L -> H -> A  and  L -> M  and  K -> H


Diagonalising Hamiltonian kblocks: 100%|████████████████| Time: 0:00:32


In [9]:
dos = plot_dos(scfres)
plot!(dos, xlabel="ε", ylabel="ρ(ε)", ylim=(0, 60));

In [16]:
summary = plot(ρ, bands, dos, layout = (1, 3), size=(800, 400), title=["ρ" "ε(k)" "ρ(ε)"])
savefig(summary, "graphene_summary.pdf")

<img src="graphene_summary.pdf" alt="drawing" width="1000" align="left"/>